In [440]:
def computeBM25Score(R,S,idf,tf,k,b,avgDL):
    S = S.split()
    R = R.split()
    score = 0
    for ws in S:
        numerator = tf[ws] * (k + 1)
        denominator = tf[ws] + k * (1- b + b * (len(R)/avgDL))
        score += computeIdf(ws,idf)  * numerator/denominator
    return score 

In [441]:
def computeIdf(word,idf):
    if word in idf:
        #print(idf[word])
        return idf[word]
    else:
        return 1

In [442]:
def readFile(path):
    file = open(path,"r")
    sentences = []
    for line in file:
        line = line.strip()
        if line.strip():
            if line == "@highlight":
                break
            sentences.append(line)
            
    file.close()
    return sentences

In [443]:
def stopping(sentences):
    stoppedSent = []
    stopWords =[]
    file = open("stop-word-list.txt", 'r')
    for line in file:
        stopWords.append(line.strip())
    for s in sentences:
        temp = s.split()
        sent = ""
        for w in temp:
            if w not in stopWords:
                sent += w + " "
        stoppedSent.append(sent)
    return stoppedSent
    

In [444]:
from nltk.stem import PorterStemmer 

def stemming(sentences):
    stemmedSent = []
    ps = PorterStemmer()
    for s in sentences:
        temp = s.split()
        stemmedWord = ""
        for w in temp:
            stemmedWord += ps.stem(w) + " "
        stemmedSent.append(stemmedWord)
    return stemmedSent
            
    

In [445]:
def createSimilarityMatrix(sentences,idf,k,b):
    BM25_scores = [[0 for x in range(len(sentences))] for y in range(len(sentences))]
    avgDL = computeAvgDL(sentences)
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i == j:
                BM25_scores[i][j] = 1
            else:
                
                tf = generateTF(sentences[i],sentences[j])
                BM25_scores[i][j] = computeBM25Score(sentences[i],sentences[j],idf,tf,k,b,avgDL)
    #print(BM25_scores)
    return BM25_scores

In [446]:
def computeAvgDL(sentences):
    sum =  0
    for s in sentences:
        sum += len(s.split())
    avg = sum/len(sentences)
    return avg

In [447]:
def generateTF(R,S):
    tf = {}
    #print(R,S)
    S = S.split()
    R = R.split()
    for ws in S:
        if ws in R:
            if ws in tf:
                tf[ws] += 1
            else:
                tf[ws] = 1
        else:
            tf[ws] = 0
    #print(tf)
    return tf

In [448]:
import random
def textRank(sentences,BM25_Score,d):
    ws = {}
    #print(BM25_Score)
    #assigning initial pagerank 
    for i in range(len(sentences)):
        ws[i] = 1/len(sentences)
    currSum = 0
    prevSum = -1
    
    count = 0
    while((currSum - prevSum) == 0 or count != 10 ):
        prevSum = currSum
        count += 1
        for i in range(len(sentences)):
            inlinkContri = 0
            for j in range(len(sentences)):
                if j == i:
                    continue
                outlinkContrib = 0
                for k in range(len(sentences)):
                    if k == j:
                        continue
                    outlinkContrib += BM25_Score[j][k]
                if outlinkContrib != 0:
                    inlinkContri += (BM25_Score[j][i]/outlinkContrib) * ws[j]
                else: 
                    inlinkContri = 0
            ws[i] =  (1-d) + d * inlinkContri
            currSum += ws[i]
    return ws

In [449]:
from os import getcwd
from os.path import join
def generateIdfDict():
    idfFile = 'finalIdfscore.csv'
    file = join(getcwd(), idfFile)
    idfDict = {}
    f = open(file,'r',encoding="utf8")
    for line in f:
        lineArr = line.split(',')
        idfDict[lineArr[0]] = float(lineArr[1])
    f.close()
    print("idf done")
    return idfDict

In [450]:
import operator
def summarize(path,outPath,ratio):
    b = 0.75
    k = 1.2
    d = 0.85
    sentences = readFile(path)
    stoppedSentences = stopping(sentences)
    stemmedSentences = stemming(sentences)
    #print(stemmedSentences)
    idf = generateIdfDict()
    BM25_Score = createSimilarityMatrix(stemmedSentences,idf,k,b)
    sentenceScore = textRank(stemmedSentences,BM25_Score,d)
    file = open(outPath,"w+")
    n = int(ratio * len(sentences))
    
    #print(sentenceScore)
    sentenceScore = dict(sorted(sentenceScore.items(), key=lambda x:x[1], reverse=True)[:n])
    sentenceScore = sorted(sentenceScore.items(), key=lambda x:x[0])
    print(sentenceScore)
    print()
    for index in sentenceScore:
        i = index[0]
        file.write(sentences[i])
        print(sentences[i])
        file.write("\n")
        print("\n")
    file.close()

In [451]:
summarize("C:\\NewsData\\NewsFile4.txt","C:\\NewsData\\SummryNewsFile4.txt",0.2)


idf done
[(3, 1.843007195591698), (9, 2.3299270808429435), (15, 1.7364141278868264), (16, 1.4718750933778582), (18, 1.7042117657409341), (21, 1.991132131721855), (23, 1.5271616223367093), (36, 2.3358843998270644), (40, 1.8243634932393884)]

Novelty nonhuman accounts have been a Twitter staple since Sockington, a Boston-area housecat, joined the service in March 2007 on his way to 1.4 million followers. But they've been in the news again recently after one user created @olympicseat, a tongue-in-cheek response to the curiously vacant seats at Olympic venues in London. In the span of a few days, it has amassed more than 21,000 followers.


"I figured the best way to test the waters was with a pseudonym," said the Burlington, Vermont, resident who asked that her name not be published to preserve the anonymity of her Twitter persona. "I wouldn't offend friends and colleagues if I didn't follow them, and I could make a polite exit from Twitter when I was done poking around."


"I love everyt